<a href="https://www.kaggle.com/code/khoshbayani/mobile-price-classification?scriptVersionId=139771509" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
import numpy as np
print("importing finished")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

importing finished


Because the test.csv doesn't have labels we can't get the accuracy of predicted labels on this dataset, So I decided to ignore "test.csv".

In [2]:
data = pd.read_csv('/kaggle/input/mobile-price-classification/train.csv')

In [3]:
data.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [4]:
data.describe()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
count,2000.000000,2000.0000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1238.518500,0.4950,1.522250,0.509500,4.309500,0.521500,32.046500,0.501750,140.249000,4.520500,...,645.108000,1251.515500,2124.213000,12.306500,5.767000,11.011000,0.761500,0.503000,0.507000,1.500000
std,439.418206,0.5001,0.816004,0.500035,4.341444,0.499662,18.145715,0.288416,35.399655,2.287837,...,443.780811,432.199447,1084.732044,4.213245,4.356398,5.463955,0.426273,0.500116,0.500076,1.118314
min,501.000000,0.0000,0.500000,0.000000,0.000000,0.000000,2.000000,0.100000,80.000000,1.000000,...,0.000000,500.000000,256.000000,5.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,851.750000,0.0000,0.700000,0.000000,1.000000,0.000000,16.000000,0.200000,109.000000,3.000000,...,282.750000,874.750000,1207.500000,9.000000,2.000000,6.000000,1.000000,0.000000,0.000000,0.750000
50%,1226.000000,0.0000,1.500000,1.000000,3.000000,1.000000,32.000000,0.500000,141.000000,4.000000,...,564.000000,1247.000000,2146.500000,12.000000,5.000000,11.000000,1.000000,1.000000,1.000000,1.500000
75%,1615.250000,1.0000,2.200000,1.000000,7.000000,1.000000,48.000000,0.800000,170.000000,7.000000,...,947.250000,1633.000000,3064.500000,16.000000,9.000000,16.000000,1.000000,1.000000,1.000000,2.250000
max,1998.000000,1.0000,3.000000,1.000000,19.000000,1.000000,64.000000,1.000000,200.000000,8.000000,...,1960.000000,1998.000000,3998.000000,19.000000,18.000000,20.000000,1.000000,1.000000,1.000000,3.000000


It doesn't contain any null value.

In [5]:
data.isna().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [6]:
data.dtypes

battery_power      int64
blue               int64
clock_speed      float64
dual_sim           int64
fc                 int64
four_g             int64
int_memory         int64
m_dep            float64
mobile_wt          int64
n_cores            int64
pc                 int64
px_height          int64
px_width           int64
ram                int64
sc_h               int64
sc_w               int64
talk_time          int64
three_g            int64
touch_screen       int64
wifi               int64
price_range        int64
dtype: object

In [7]:
data['price_range'].unique()

array([1, 2, 3, 0])

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,:-1],data['price_range'], test_size=0.3)

The scale of our dataset is not large, So it is predictable that neural networks don't have a good performance on this dataset.
But everybody wants to see it with his/her eyes.

In [9]:
#------------------------------------------------------
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
#------------------------------------------------------
from sklearn.metrics import accuracy_score

svm = SVC()
gaussianNB= GaussianNB()
decisionTreeClassifier = DecisionTreeClassifier()
kneighborsClassifier = KNeighborsClassifier()
randomForestClassifier= RandomForestClassifier()


for alg in [svm, gaussianNB, decisionTreeClassifier, kneighborsClassifier, randomForestClassifier]:
    print(str(alg)[:-2])
    alg.fit(X_train,y_train)
    print('acc on train:',alg.score(X_train,y_train))
    print('acc on test:',alg.score(X_test,y_test))
    print('--------------------------')


SVC
acc on train: 0.9478571428571428
acc on test: 0.9566666666666667
--------------------------
GaussianNB
acc on train: 0.8121428571428572
acc on test: 0.8216666666666667
--------------------------
DecisionTreeClassifier
acc on train: 1.0
acc on test: 0.8466666666666667
--------------------------
KNeighborsClassifier
acc on train: 0.9442857142857143
acc on test: 0.9483333333333334
--------------------------
RandomForestClassifier
acc on train: 1.0
acc on test: 0.8983333333333333
--------------------------


As we can see the best score (with default parameters) has been got from **SVM**.

Now lest's search for better parameters:

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import SVC
import numpy as np
svm = SVC()
params={
    'C':np.arange(1,10,1),
    'kernel':['sigmoid', 'poly', 'rbf','linear'],
}

cv = ShuffleSplit(n_splits=5,test_size=0.25,random_state=1234)
clf=GridSearchCV(
    svm,
    param_grid=params,
    scoring='accuracy',
    cv=cv,
    verbose=10,
    return_train_score=True,

)

clf_r = clf.fit(X_train,y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5; 1/36] START C=1, kernel=sigmoid........................................
[CV 1/5; 1/36] END C=1, kernel=sigmoid;, score=(train=0.182, test=0.151) total time=   0.1s
[CV 2/5; 1/36] START C=1, kernel=sigmoid........................................
[CV 2/5; 1/36] END C=1, kernel=sigmoid;, score=(train=0.185, test=0.129) total time=   0.1s
[CV 3/5; 1/36] START C=1, kernel=sigmoid........................................
[CV 3/5; 1/36] END C=1, kernel=sigmoid;, score=(train=0.170, test=0.154) total time=   0.1s
[CV 4/5; 1/36] START C=1, kernel=sigmoid........................................
[CV 4/5; 1/36] END C=1, kernel=sigmoid;, score=(train=0.178, test=0.206) total time=   0.1s
[CV 5/5; 1/36] START C=1, kernel=sigmoid........................................
[CV 5/5; 1/36] END C=1, kernel=sigmoid;, score=(train=0.171, test=0.143) total time=   0.1s
[CV 1/5; 2/36] START C=1, kernel=poly....................................

In [11]:
print(clf_r.best_params_)


{'C': 1, 'kernel': 'linear'}


The result:

In [12]:
from sklearn.metrics import accuracy_score


y_pred_on_train = clf_r.predict(X_train)
y_pred_on_test = clf_r.predict(X_test)

print("result:")
print('acc on train: ',accuracy_score(y_train,y_pred_on_train))
print('acc on test: ',accuracy_score(y_test,y_pred_on_test))

result:
acc on train:  0.9942857142857143
acc on test:  0.9733333333333334
